In [1]:
import time
import math
from random import randint

primes_20 = [826151, 624401, 701653, 557159, 677227, 895681, 891851, 563777, 623321, 708907]
primes_224 = [19529360954299272297074890573062984085485619732672344646176777051089, 13803237858330031352733408675178090680573238167361367485172686664551, 24619895622997953504019111802747844952261235752777751456173932309577, 15756374287552202866875426879367027788654168281775352416831798751199, 24197832980950551884321297696074026865588478848759402739855797333271, 13626745861916852426339692888159817894242378055350580135154320867093, 23521206951626368959318339376803040067462477302963916539056520318637, 23339266892146323971605413070894572286348238274753659032136801386337, 25704212596991814333920455677097318297258901714981631987809888960147, 24848234262474898966674262065219920517132212746019962652209091500021]
primes_256 = [68507249821448912525837317006796370923008574932957992418268086538574862869691, 77870379314126793251255353291633818861279973088911985044901529454516883270803, 91464074189092445410652975601482604937471946510377367038810080054255282595963, 79474397695743613709972446957117208576120338365608281173866213462940550477073, 63642325730707692168414031399847641048898182053136139903169810457936310089853, 81152978723497138317253297930199972353458087836352467762922030236205448609103, 85646962083865094308029130740740408225629105444082513396528162917864345247153, 82230190471146640256031429777336144531399742954902919944301250353094232800369, 70409199191241335629231529205201962044321344385335507620302963304913744302281, 78853106220787152860227756611206954920169805732063339965344977048702892540633]
primes_288 = [254912771143371719495952320420689424577240582815611191134928873052667553587389831700953, 439800259142823478703484628522557709001989904683258120096591813819139384829007565387611, 330577885750003981680156866513758676354049783089135541108002971023420668059538753582837, 372014207726558411156528835713250523332111066396759275611097116160868959897631562709659, 485054805086785805441803947205955947088394008434249092107030733863860845530351462595359, 302817259074339098293160986486635357917430550037847744911311509621963881007255249211007, 455844996029346660659261004729905087001749292535192413469514767388354509759978142211947, 343746328922129646338215433259500635316453944897196560449812049549246071134611521762173, 329943488955159248694050152298141764421776817499037589700011535998729818745732023421497, 357357154847897813271155216309459801536722353019206037278276847717741465245247614698309]

# returnes matrix mu and list of squared euclidean norms
#   of the vectors in Gram-Schmidt orthogonalized basis
#   (Cholesky decomposition avoiding use of irrationals)
# G ... Gram matrix to be decomposed
def chol_decomp(G, cols=-1):
    rows = G.nrows()
    if cols < 0:
        cols = rows
    mu = matrix(QQ, rows, cols)
    squares = vector(QQ, cols)
    for y in range(rows):
        over_cols = (y>=cols)
        for x in range(cols if over_cols else y):
            t = 0
            for i in range(x):
                t += mu[y,i]*mu[x,i]*squares[i]
            mu[y,x] = (G[y,x]-t)/squares[x]
        if not over_cols:
            t = 0
            for i in range(y):
                t += mu[y,i]*mu[y,i]*squares[i]
            mu[y,y] = 1
            squares[y] = G[y,y]-t
    return (mu, squares)

# swaps i-th row with the previous one
def swap_rows_in_mu(mu, s, i, n):
    mu[i-1], mu[i] = mu[i], mu[i-1]
    mu[i,i], mu[i-1,i] = 1, 0
    r = mu[i-1,i-1]
    mu[i-1,i-1] = 1
    temp = s[i-1]*r**2 + s[i]
    mu[i,i-1] = r*s[i-1]/temp
    s[i] = s[i-1]*s[i]/temp
    s[i-1] = temp
    for j in range(i+1, n):
        mu[j,i-1], mu[j,i] = mu[i,i-1]*mu[j,i-1]+mu[j,i]*(1-r*mu[i,i-1]), \
                           mu[j,i-1]-r*mu[j,i]

# auxiliary function: the inner cycle of LLL
def LLL_reduce_row(B, mu, i, h):
    for j in range(h-1, -1, -1):
        t = round(mu[i, j])
        for k in range(j+1):
            mu[i, k] -= t*mu[j, k]
        B[i] -= t*B[j]

# auxiliary LLL reduction algorithm
# B ... integer basis (does'n have to be square matrix)
# mu ... Gram-Schmidt mu matrix (doesn't have to be square matrix)
# s ... squared norms of Gram-Schmidt orthogonalized vectors
# n ... number of rows of B to be reduced (usually B.nrows())
# h ... index of first row not to be swapped (only reduced)
def LLL_aux(B, delta, mu, s, n, h):
    i = 1
    flag = True
    while i < h:
        if flag:
            LLL_reduce_row(B, mu, i, i)
        if i > 0 and (delta-mu[i,i-1]**2)*s[i-1] > s[i]:
            B[i], B[i-1] = B[i-1], B[i]
            swap_rows_in_mu(mu, s, i, n)
            i -= 1
            flag = False
        else:
            i += 1
            flag = True
    while i < n:  # performed only if n>h
        LLL_reduce_row(B, mu, i, h)
        i += 1
    return B, mu, s

# implementation of classical LLL algorithm
# last row of B can be used as a point in Babai's algorithm
def LLL_own(B, delta=3/4, babai=False):
    mu, s = chol_decomp(B*B.transpose(), B.nrows()-babai)
    return LLL_aux(copy(B), delta, mu, s, B.nrows(), B.nrows()-babai)[0]

# LLL for triangular input basis
def LLL_triangular_input(B, delta=3/4, babai=False):
    s = [B[j,j]**2 for j in range(B.ncols())]
    mu = copy(B)
    for j in range(mu.ncols()):
        r = mu[j,j]
        for i in range(j, mu.nrows()):
            mu[i,j] /= r
    return LLL_aux(copy(B), delta, mu, s, B.nrows(), B.nrows()-babai)[0]

# returned matrix will not be unimodular
#   if v is not primitive (coprime)
def complete_row_to_unimodular(v):
    n = len(v)
    ret = matrix(ZZ, n, n)
    ret[0] = v
    d = v[0]
    for i in range(1,n):
        tup = xgcd(d, v[i])
        ret[i,i] = tup[1]
        for j in range(i):
            ret[i,j] = -tup[2]*ret[0,j]//d if d else 1
        d = tup[0]
    return ret

# implementation of modified Fisher-Yates algorithm
# returns random permutation of m-member random subset of s
#   (whole set is taken if m is smaller than zero)
def random_permutation(s, m=-1):
    n = len(s)
    m = n if m<0 else m
    sig = list(range(n))
    ret = []
    for i in range(m):
        j = randint(i, n-1)
        ret.append(s[sig[j]])
        sig[j] = sig[i]
    return ret

# absolute value of centered modulo of integers
def abs_mod(a, m):
    ret = a % m
    return (ret if ret <= m//2 else m-ret)

# absolute value of centered modulo of real numbers
def abs_mod_float(a, m):
    return abs(a-m*round(a/m))

# modulo of real numbers
def mod_float(a, m):
    return a-m*math.floor(a/m)

# inversion of a (mod n)
def mod_inversion(a, n):
    a, m, k1, k2 = a%n, n, 0, 1
    while a!=0:
        r, d = n%a, n//a
        k1, k2, n, a = k2, k1-d*k2, a, r
    return (k1%m if n==1 else None)

# list_A and list_B must contain disjoint intervals in increasing order
# open intervals are used (endpoints not included)
# inspired by
#   https://geeksforgeeks.org/find-intersection-of-intervals-given-by-two-lists
def intersect_two_interval_lists(list_A, list_B):
    ret = []
    i, j, n, m = 0, 0, len(list_A), len(list_B)
    while i < n and j < m:
        low = max(list_A[i][0], list_B[j][0])
        if list_A[i][1] <= list_B[j][1]:
            top = list_A[i][1]
            i += 1
        else:
            top = list_B[j][1]
            j += 1
        if low < top:
            ret.append((low, top))
    return ret

# list_of_lists: list whose elements are lists of
#   disjoint intervals in increasing order
# returned value is a list of disjoint intervals in increasing order
def intersect_interval_lists(list_of_lists):
    ret = [(-math.inf, math.inf)]
    for alist in list_of_lists:
        ret = intersect_two_interval_lists(ret, alist)
    return ret

# creates a list of all intervals containing real solutions
#   to one HNP inequality
# covers all solutions between 0 and q
# some of the returned intervals can possibly exceed this range
def make_interval_list_HNP(q, ti, ui, w):
    period = q/abs(ti)
    b = mod_float(ui/ti + period/(2*w), period)
    a = b - period/w
    k, cur, ret = 0, a, []
    while cur < q:
        ret.append((cur, cur+period/w))
        k += 1
        cur = a + k*period
    return ret

# uses <=, not <
# returns a pair (alpha, overflows); alpha is the solution
# ti, ui, ri must be between 0 and q-1 (inclusive)
def decentered_ineq_least_solution(q, ti, ui, ri):
    if (-ui)%q <= ri:
        return (0, 0)
    if ti==0:
        raise ValueError  # no solution exists
    rev = False
    if 2*ti > q:
        ui = (-ui-ri)%q
        ti = q-ti
        rev = True
    inner_cost = ceil(q/ti)
    init_jumps = ceil(ui/ti)
    if (init_jumps*ti-ui)%q <= ri:
        return (init_jumps, 0)
    init_jumps -= 1
    inner_jumps, inner_overflows = \
            decentered_ineq_least_solution(ti, inner_cost*ti-q,
                                           ui-init_jumps*ti, ri)
    jumps = init_jumps + inner_jumps*inner_cost - inner_overflows
    overflows = (jumps*ti)//q
    if rev:
        overflows = jumps - 1 - overflows
    return jumps, overflows


class HNP:
    def __init__(self, q, default=None, li_vs_wi=True,
                 matrix_B_corner=1, alpha=None):
        self.q = q  # should be prime number, not checked
        self.alpha = alpha
        self.d = 0
        self.t = []
        self.u = []
        self.li_vs_wi = li_vs_wi  # using number of "bits" (li) is default
        if li_vs_wi:
            self.default = default if default else 1
        else:
            self.default = default if default else 2
        self.l_or_w_dict = {}
        self.interval_list = None
        self.number_of_intervals = None
        self.solution_list = None
        self.number_of_solutions = None
        self.matrix_B_corner = matrix_B_corner
        self.reduced_HNP = None
    
    def get_w(self, i=-1):
        if self.li_vs_wi:
            return 2**self.l_or_w_dict.get(i, self.default)
        return self.l_or_w_dict.get(i, self.default)
    
    def clear(self):  # back to initial state
        self.__init__(self.q, self.default, self.li_vs_wi,
                      self.matrix_B_corner, self.alpha)
        return self
    
    def sort_by_t(decreasing=False):
        self.t, self.u = zip(*sorted(zip(t,u),
                         key=(lambda tup: abs_mod(tup[0], self.q)),
                         reverse=decreasing))
    
    def make_reduced_HNP(self):
        reduced_HNP = HNP(self.q, self.default,
                          self.li_vs_wi, (self.q-1)//((self.q-1)//(2*self.get_w())))
        reduced_HNP.l_or_w_dict = copy(self.l_or_w_dict)
        t_1_1 = mod_inversion(self.t[-1], self.q)
        temp = (-self.u[-1]*t_1_1)%self.q
        for i in range(self.d-1):
            reduced_HNP.add_ineq((t_1_1*self.t[i])%self.q,
                                       (self.u[i]+temp*self.t[i])%self.q)
        self.reduced_HNP = reduced_HNP
        return self.reduced_HNP
    
    def solution_from_reduced_HNP(self):
        return ((self.reduced_HNP.alpha+self.u[-1]) \
                 *mod_inversion(self.t[-1],self.q))%self.q
    
    def solutions_from_reduced_HNP(self):
        ret = []
        for sol in self.reduced_HNP.solution_list:
            ret.append(((sol+self.u[-1])* \
                        mod_inversion(self.t[-1],self.q))%self.q)
        return ret
    
    def make_random(self, n=-1):
        self.clear()
        wi = 2**self.default if self.li_vs_wi else self.default
        if n < 0:
            s, n = 1, 0
            while s < self.q:
                s *= wi
                n += 1
        self.alpha = self.alpha if self.alpha else randint(1,self.q-1)
        self.t = [randint(1,self.q-1) for _ in range(n)]
        self.u = [(self.alpha*self.t[i]+randint(math.floor(-self.q/(2*wi)+1),
                  math.ceil(self.q/(2*wi)-1)))%self.q for i in range(n)]
        self.d = n
        return self
    
    def check_solution(self, int_number):
        return all([abs_mod(int_number*self.t[i]-self.u[i], \
                    self.q)*2*self.get_w(i) < self.q for i in range(self.d)])
    
    def check_solution_float(self, float_number):
        return all([abs_mod_float(float_number*self.t[i]-self.u[i], \
                    self.q)*2*self.get_w(i) < self.q for i in range(self.d)])
    
    def make_matrix_B(self):
        return matrix([[(2*self.get_w(i)*self.q if i==j else 0) \
                        for j in range(self.d+1)] \
                       for i in range(self.d)] + \
                [[2*self.get_w(j)*self.t[j] for j in range(self.d)] + \
                 [self.matrix_B_corner]])
    
    def make_matrix_C(self):  # C is called B' in the thesis
        return matrix([[(2*self.get_w(i)*self.q if i==j else 0) \
                        for j in range(self.d+2)] \
                       for i in range(self.d)] + \
            [[2*self.get_w(j)*self.t[j] for j in range(self.d)]+ \
             [self.matrix_B_corner, 0]] + \
            [[2*self.get_w(j)*self.u[j] for j in range(self.d)]+[0, self.q]])
    
    def make_matrix_B_half(self):
        return matrix([[(self.get_w(i)*self.q if i==j else 0) \
                        for j in range(self.d+1)] \
                       for i in range(self.d)] + \
                [[self.get_w(j)*self.t[j] for j in range(self.d)] + \
                 [self.matrix_B_corner]])
    
    def make_matrix_C_half(self):  # C is called B' in the thesis
        return matrix([[(self.get_w(i)*self.q if i==j else 0) \
                        for j in range(self.d+2)] \
                       for i in range(self.d)] + \
            [[self.get_w(j)*self.t[j] for j in range(self.d)]+ \
             [self.matrix_B_corner, 0]] + \
            [[self.get_w(j)*self.u[j] for j in range(self.d)]+[0, self.q]])
    
    # creates a basis equivalent to matrix C (called B' in the thesis)
    #   having first row (0, 0,..., 0, q, 0)
    def make_matrix_C_with_first_row_set(self):
        U = complete_row_to_unimodular(
              vector([-self.t[i] for i in range(self.d)]+[self.q, 0]))
        return U * self.make_matrix_C()
    
    def add_ineq(self, ti, ui, li_or_wi=None):
        if li_or_wi and li_or_wi!=self.default:
            self.l_or_w_dict[self.d] = li_or_wi
        self.t.append(ti)
        self.u.append(ui)
        self.d += 1
        return self
    
    def pop_ineq(self):
        self.d -= 1
        return (self.t.pop(), self.u.pop(),
                self.l_or_w_dict.pop(self.d, self.default))
    
    def make_intervals(self, find_integers=True):
        self.interval_list = intersect_interval_lists( \
            [make_interval_list_HNP(self.q, self.t[i], self.u[i], self.get_w(i)) \
             for i in range(self.d)]+[[(0,self.q)]])
        self.number_of_intervals = len(self.interval_list)
        if self.interval_list[0][0]==0 and self.interval_list[-1][1]==self.q \
                and self.check_solution(0):
            self.number_of_intervals -= 1
        if find_integers:
            self.integer_solution_from_intervals()
        return self
    
    def bruteforce_integer_solution(self):
        ret = []
        for i in range(self.q):
            if self.check_solution(i):
                ret.append(i)
        if self.solution_list is None:
            self.number_of_solutions = len(ret)
            self.solution_list = ret
        else:
            assert self.number_of_solutions == len(ret)
            assert self.solution_list == ret
        return self
    
    def integer_solution_from_intervals(self):
        assert self.interval_list
        ret = []
        if self.check_solution(0):
            ret.append(0)
        for r in self.interval_list:
            ret.extend(range(math.floor(r[0]+1), math.ceil(r[1])))
        if self.solution_list is None:
            self.number_of_solutions = len(ret)
            self.solution_list = ret
        else:
            assert self.number_of_solutions == len(ret)
            assert self.solution_list == ret
        return self


In [4]:
# compared methods
STANDARD_BABAI_75 = 0
STANDARD_BABAI_99 = 1
HALF_BABAI_75 = 2
HALF_BABAI_99 = 3
STANDARD_LLL_OWN_75 = 4
STANDARD_LLL_OWN_99 = 5
HALF_LLL_OWN_75 = 6
HALF_LLL_OWN_99 = 7
STANDARD_LLL_75 = 8
STANDARD_LLL_99 = 9
HALF_LLL_75 = 10
HALF_LLL_99 = 11
STANDARD_BKZ_5 = 12
STANDARD_BKZ_10 = 13
CHANGED_LLL_75 = 8
CHANGED_LLL_99 = 9
CHANGED_HALF_LLL_75 = 10
CHANGED_HALF_LLL_99 = 11
CHANGED_BKZ_5 = 12
CHANGED_BKZ_10 = 13
REDUCED_HALF_BABAI_75 = 14
REDUCED_HALF_BABAI_99 = 15
INCREASING_BABAI_75 = 16
DECREASING_BABAI_75 = 17
# maybe more...

table_224 = {}
#table_256 = {}
#table_288 = {}
primes_224_perm = random_permutation(primes_224)
#primes_256_perm = random_permutation(primes_256)
#primes_288_perm = random_permutation(primes_288)
c = 0

primes_list = primes_224_perm
for q in primes_list:
    table_224[q] = {}
    bit_length = len(bin(q))-2
    for li in [7, ceil(sqrt(bit_length))+ceil(log(bit_length,2))]:
        table_224[q][li] = {}
        for d in [ceil(bit_length/li),
                  ceil(6/5*bit_length/li),
                  ceil(2*bit_length/li)]:
            table_224[q][li][d] = {}
            table_224[q][li][d][STANDARD_BABAI_75] = []
            table_224[q][li][d][STANDARD_BABAI_99] = []
            #table_224[q][li][d][XXX] = []
            for _ in range(3):
                prob = HNP(q, li)
                prob.make_random(d)
                start = time.time()
                C = prob.make_matrix_C()
                red = LLL_own(C[:, :C.ncols()-1], 75/100, babai=True)
                cand_sol = -red[-1, -1]%prob.q
                stop = time.time()
                table_224[q][li][d][STANDARD_BABAI_75].append((\
                        stop-start,
                        prob.check_solution(cand_sol),
                        cand_sol==prob.alpha))
                print(c, end=" ")
                c += 1
            for _ in range(3):
                prob = HNP(q, li)
                prob.make_random(d)
                start = time.time()
                C = prob.make_matrix_C()
                red = LLL_own(C[:, :C.ncols()-1], 99/100, babai=True)
                cand_sol = -red[-1, -1]%prob.q
                stop = time.time()
                table_224[q][li][d][STANDARD_BABAI_99].append((\
                        stop-start,
                        prob.check_solution(cand_sol),
                        cand_sol==prob.alpha))
                print(c, end=" ")
                c += 1

#primes_list = primes_256_perm
# TO DO!!!

#primes_list = primes_288_perm
# TO DO!!!

print(table_224)


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 

KeyboardInterrupt: 

In [5]:
print(table_224)

{23339266892146323971605413070894572286348238274753659032136801386337: {7: {32: {0: [(19.714274168014526, False, False), (19.27436065673828, False, False), (18.637360095977783, False, False)], 1: [(52.50889468193054, False, False), (52.571044921875, False, False), (51.57638168334961, False, False)]}, 39: {0: [(30.295044422149658, False, False), (30.890860319137573, False, False), (30.794458389282227, False, False)], 1: [(85.78024101257324, False, False), (87.6913435459137, False, False), (100.35513758659363, True, True)]}, 64: {0: [(98.42664384841919, True, True), (95.81256484985352, False, False), (107.17437100410461, True, True)], 1: [(325.5675127506256, True, True), (338.2974519729614, True, True), (327.10998153686523, True, True)]}}, 23: {10: {0: [(1.0509419441223145, True, True), (1.158597469329834, True, True), (1.120347499847412, True, True)], 1: [(1.9200587272644043, True, True), (1.8722975254058838, True, True), (1.7635259628295898, True, True)]}, 12: {0: [(1.7286138534545898,

In [11]:
prob2 = HNP(primes_256[0], 17)
prob2.make_random(17)
print(prob2.alpha)
C2 = prob2.make_matrix_C()
start = time.time()
red = LLL_own(C2[:, :C2.ncols()-1], 75/100, babai=True)
number_found = -red[-1, -1]%prob2.q
print(number_found)
print(number_found==prob2.alpha)
stop = time.time()
print("running time was: "+str(stop-start))

29217901602643454489226402204939365223673664465223249243090284675682054272727
29217901602643454489226402204939365223673664465223249243090284675682054272727
True
running time was: 4.975225210189819


In [13]:
C2_half = prob2.make_matrix_C_half()
start = time.time()
red_half = LLL_own(C2_half[:, :C2_half.ncols()-1], 75/100, babai=True)
number_found = -red_half[-1, -1]%prob2.q
print(number_found)
print(number_found==prob2.alpha)
stop = time.time()
print("running time was: "+str(stop-start))

29217901602643454489226402204939365223673664465223249243090284675682054272727
True
running time was: 4.9088544845581055


In [18]:
start = time.time()
C_child = prob2.make_reduced_HNP().make_matrix_C_half()
red_half = LLL_own(C_child[:, :C_child.ncols()-1], 75/100, babai=True)
prob2.reduced_HNP.alpha = -red_half[-1, -1]%prob2.q
print(number_found)
print(number_found==prob2.alpha)
stop = time.time()
print("running time was: "+str(stop-start))

29217901602643454489226402204939365223673664465223249243090284675682054272727
True
running time was: 3.7099287509918213


In [4]:
start = time.time()
prob2_red = prob2.make_reduced_HNP()
C2_red = prob2_red.make_matrix_C()
#print(C2_red)
red_red = LLL_own(C2_red[:, :C2_red.ncols()-1], 75/100, babai=True)
prob2_red.alpha = -red_red[-1, -1]%prob2_red.q//prob2_red.matrix_B_corner
number_found_red = prob2.solution_from_reduced_HNP()
print(number_found_red)
print(number_found_red==prob2.alpha)
stop = time.time()
print("running time was: "+str(stop-start))

40467679157867726705875985605498144524938128917044906277880020482454850793430
False
running time was: 3.730813980102539


In [38]:
print([abs(red[-1, i])<prob2.q for i in range(red.ncols())])

[True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True]


In [40]:
start = time.time()
prob2 = HNP(primes_256[0], 17)
prob2.make_random(15)
print(prob2.alpha)
C2 = prob2.make_matrix_C()
red = LLL_own(C2[:, :C2.ncols()-1], 75/100, babai=True)
number_found = -red[-1, -1]%prob2.q
print(number_found)
print(number_found==prob2.alpha)
stop = time.time()
print("running time was: "+str(stop-start))

33703159984191327545405271965097094832127957336051849962853494544199297844209
33703159984191327545405271965097094832127957336051849962853494544199297844209
True
running time was: 3.417381763458252


In [41]:
print([abs(red[-1, i])<prob2.q for i in range(red.ncols())])

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
